In [ ]:
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi

In [ ]:
%reload_ext tensorboard

In [1]:
import torch
import torch.nn as nn
from torchaudio import load, transforms
from datasets import load_dataset, load_metric, load_from_disk
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, logging as log_models
from datasets import Dataset, DatasetDict, set_caching_enabled, logging as log_data
from torch.utils.tensorboard import SummaryWriter

import json
import tempfile
import os
import gc
import re
import pickle
import pandas as pd
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
from  IPython.display import clear_output
from datetime import datetime
from tqdm.notebook import tqdm

from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
# torch.backends.cudnn.enabled = False
torch.backends.cudnn.deterministic = True

import nltk
nltk.download('averaged_perceptron_tagger')

# from google.colab import drive
# drive.mount('/content/drive')

from profiler import ProbingProfiler
from constants import *

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
TODAY = str(datetime.now().strftime('%Y-%m-%d')) 
!mkdir cache
!mkdir checkpoints
!mkdir jsons
!mkdir tensorboards

!mkdir tensorboards/profiling
!mkdir checkpoints/$TODAY
!mkdir jsons/$TODAY

CACHE_DIR = './cache'

mkdir: cannot create directory ‘cache’: File exists
mkdir: cannot create directory ‘checkpoints’: File exists
mkdir: cannot create directory ‘jsons’: File exists
mkdir: cannot create directory ‘tensorboards’: File exists
mkdir: cannot create directory ‘tensorboards/profiling’: File exists
mkdir: cannot create directory ‘checkpoints/2022-02-01’: File exists
mkdir: cannot create directory ‘jsons/2022-02-01’: File exists


In [3]:
POOLING_TO = 4
def kl_divergence(z, mu_theta, p_theta):
    log_prior = torch.distributions.Normal(0, 1).log_prob(z) 
    log_p_q = torch.distributions.Normal(mu_theta, torch.log(1 + torch.exp(p_theta))).log_prob(z) 
    return (log_p_q - log_prior).mean()
 
class KL:
    accumulated_kl_div = 0

class Loss:
    def __init__(self, variational: bool = True):
        self.variational = variational
    def __call__(self, y_true, y_pred, model = None):
        if self.variational:
            reconstruction_error = torch.nn.CrossEntropyLoss()(y_pred, y_true)
            kl = model.accumulated_kl_div
            model.reset_kl_div()
            return reconstruction_error + kl
        else: return torch.nn.CrossEntropyLoss()(y_pred, y_true)

class LinearVariational(torch.nn.Module):
    def __init__(self, in_features: int, out_features: int, parent, bias: bool=True, device = device):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.device = device
        self.include_bias = bias        
        self.parent = parent
        
        if getattr(parent, 'accumulated_kl_div', None) is None:
            if getattr(parent.parent, 'accumulated_kl_div', None) is None: parent.accumulated_kl_div = 0
            else: parent.accumulated_kl_div = parent.parent.accumulated_kl_div
            
        self.w_mu = nn.Parameter(torch.FloatTensor(in_features, out_features).normal_(mean = 0, std = 0.001).to(self.device))
        self.w_p = nn.Parameter(torch.FloatTensor(in_features, out_features).normal_(mean = 0, std = 0.001).to(self.device))

        if self.include_bias:
            self.b_mu = nn.Parameter(torch.zeros(out_features))
            self.b_p = nn.Parameter(torch.zeros(out_features))

    def _reparameterize(self, mu, p):
        sigma = torch.log(1 + torch.exp(p))
        eps = torch.randn_like(sigma)
        return mu + (eps * sigma)

    def forward(self, x):
        w = self._reparameterize(self.w_mu, self.w_p)
        
        if self.include_bias: b = self._reparameterize(self.b_mu, self.b_p)
        else: b = 0
            
        z = torch.matmul(x,w) + b
        
        self.parent.accumulated_kl_div += kl_divergence(w, self.w_mu, self.w_p).item()
        if self.include_bias: self.parent.accumulated_kl_div += kl_divergence(b, self.b_mu, self.b_p).item()
        return z

class LinearModel(torch.nn.Module):
    def __init__(self, in_size: int, hidden_size: int, out_size: int, variational: bool = True, device = device):
        super().__init__()
        self.kl_loss = KL
        self.variational = variational
        if self.variational: self.layers = torch.nn.Sequential(LinearVariational(in_size, out_size, self.kl_loss, device))
        else: self.layers = torch.nn.Sequential(torch.nn.Linear(in_size, out_size))

    @property
    def accumulated_kl_div(self):
        # assert self.variational
        return self.kl_loss.accumulated_kl_div
    
    def reset_kl_div(self):
        # assert self.variational
        self.kl_loss.accumulated_kl_div = 0
            
    def forward(self, x):
        return self.layers(x)

class ProberModel(torch.nn.Module):
    def __init__(self, parent_model, clf, enable_grads: bool):
        super().__init__()

        self.parent_model = parent_model
        self.pooling_layer = torch.nn.AdaptiveAvgPool1d(output_size = POOLING_TO)
        self.clf = clf
        self.enable_grads = enable_grads
    def forward(self, inp, att):
        parent_out = lambda e_inp, e_att: self.parent_model(e_inp, attention_mask = e_att,
                                                            output_hidden_states = False, output_attentions = False)
        if self.enable_grads: out = parent_out(inp, att)
        else: 
            with torch.no_grad(): out = parent_out(inp, att)
        out = self.pooling_layer(out.last_hidden_state.transpose(1, 2)).transpose(1, 2)
        out = self.clf(out.reshape(out.size(0), -1))
        return out

In [4]:
BATCH_SIZE = 50
N_EPOCHS = 10
MAX_LEN = 13000 #for linguistic dataset
DEBUG = False 
PROFILING = False 

GRAPHS_PATH = f"/content/drive/MyDrive/Tasks/probing_datasets/graphs/"
TENSORS_PATH = f"/content/drive/MyDrive/Tasks/probing_datasets/data/"
DATASETS_PATH = {"common_voice": f"/content/drive/MyDrive/Tasks/probing_datasets/tensors/common_voice_", 
                 "timit_asr": f"/content/drive/MyDrive/Tasks/probing_datasets/tensors/timit_asr_"}


MODELS_PATH ={"common_voice": {"ru": "anton-l/wav2vec2-large-xlsr-53-russian",
                               "fr": "facebook/wav2vec2-large-xlsr-53-french",
                               "de": "facebook/wav2vec2-large-xlsr-53-german",
                               "es": "facebook/wav2vec2-large-xlsr-53-spanish"},
              "timit_asr": {"None": "elgeish/wav2vec2-large-lv60-timit-asr"}}
TIMIT_METADATA_PATH = f"./timit_features_proc.csv"

LOGGING_DIR = "./tensorboards/"
GRAPHS_PATH = "./jsons/"
CHECKPOINTING_DIR = "./"
PROFILING_DIR = "./tensorboards/profiling/"

print_if_debug = lambda x, flag: print(x) if flag else None

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer): return int(obj)
        elif isinstance(obj, np.floating): return float(obj)
        elif isinstance(obj, np.ndarray): return obj.tolist()
        return json.JSONEncoder.default(self, obj)

class CheckPointer:
    def __init__(self, parent_dir: str): 
        """
        Class to save necessary model checkpoints and load them
        -> saving: full model, layer index, optimizer state dict 
        """
        assert isinstance(parent_dir, str) and parent_dir.endswith("/")
        self.parent_dir = os.path.join(parent_dir, "checkpoints", str(TODAY))

    def __call__(self, probing_model: torch.nn.Module, task_title: str,
                       params: dict, layer_idx: int, optimizer: torch.optim):
        """Classical torch checkpointing callable method
        Args:
            probing_model, torch.nn.Module: probing model to save
            params, dict: model configuration for inference

            layer_idx, int: index of hidden layer of the model
            optimizer, torch.optim: optional optimizer to save
        Returns:
            str, name of saved checkpoint
        checkpoint name format: CHKPNT_lyridx=int_DATETIME.pth
        """
        torch.save({
            'layer_index': layer_idx,
            'model_params': params,
            'model_state_dict': probing_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, 
            os.path.join(self.parent_dir, "probing_checkpoint_{}_lyridx={}.pth".format(task_title, layer_idx)))
        return self.parent_dir + "checkpoints/{}/probing_checkpoint_{}_lyridx={}.pth".format(datetime.now().strftime('%Y-%m-%d'),
                                                                                             task_title,
                                                                                             layer_idx)

class Prober:
    def __init__(self, model_path: str, writer: torch.utils.tensorboard.SummaryWriter, data: Dataset = None, device: torch.device = device, init_strategy: str = None) -> None:
        """ Probing tasks class.
        Args:
            model_path, str: path to model in Hugging Face repo
            data, Dataset: optional, Hugging Face Dataset class 
                           default = None
            device: torch.device, accelerator
            pretrained: str, flag (use randomly initialized model or downloaded from repo)
                             supported strategies: 
                                -- full 
                                -- (only) encoder
                                -- (only) feature_extractors
                                default = None
        """
        print_if_debug("downloading staff...", DEBUG)

        self.model = Wav2Vec2ForCTC.from_pretrained(model_path, cache_dir = CACHE_DIR).to(device)
        if init_strategy is not None:
            print_if_debug("reseting network parameters...", DEBUG)
            assert isinstance(init_strategy, str)
            if init_strategy == "full": self.model.init_weights()
            elif init_strategy == "encoder":
                for p in self.model.wav2vec2.encoder.parameters(): torch.nn.init.normal_(p)
            elif init_strategy == "feature_extractors": 
                for p in self.model.wav2vec2.feature_extractor.parameters(): torch.nn.init.normal_(p)
                for p in self.model.wav2vec2.feature_projection.parameters(): torch.nn.init.normal_(p)
            else: print("No init with {} strategy".format(init_strategy))

        self.model.freeze_feature_extractor()
        self.fixed_encoder = self.model.wav2vec2.encoder.layers.cpu()

        #debugging tools
        self.writer = writer
        
        self.checkpointer = CheckPointer(CHECKPOINTING_DIR)
        
        self.profiler = ProbingProfiler(PROFILING_DIR)
        self.profiler.on() if PROFILING else self.profiler.off() 
        self.profiler.profile()
    

        self.data = data
        self.device = device
    def get_resources(self, load_data: bool = False, data_path: str = None, batch_size: int = 100, 
                      poisoning_ratio: float = 0, poisoning_mapping: type(lambda x: None) = None, **kwargs):
        """
        Args:
          load_data, bool: optional flag, whether load data from external resources or not, ONLY DISK MODE SUPPORTED;
                           default = False
          data_path, str: optional, active only if load_data = True;
                          default = None
          batch_size, int: optional;
                           default = 100
          poisoning_ratio, float: the ratio of adding misleading labels to the data (0 -- None, 1 -- fully random);
                           default = 0.       
          poisoning_mapping, callable: the mapping of poisoned labels,
                                       default = None
        """
        print_if_debug("collecting data...", DEBUG)
        
        def poison_data(batch, n_classes: int, ratio: float = 0.01, mapping = None):
            """Adding misleading labels"""
            assert ratio > 0. and ratio <= 1.
            if np.random.random() < ratio:
                if mapping is not None: batch['label'] = mapping(batch['label']) 
                else: batch['label'] = np.random.randint(0, n_classes)
            return batch        
        
        if load_data: 
            assert isinstance(data_path, str)
            self.data =  load_from_disk(data_path, **kwargs)
        
        if poisoning_ratio > 0.: 
            self.data = self.data.map(poison_data, fn_kwargs = {"n_classes": np.max(self.data['label']),
                                                                                    'ratio': poisoning_ratio,
                                                                                     'mapping': poisoning_mapping})
            
        self.data.set_format(type = 'torch', columns = ['input_values', 'attention_mask', 'label'])

        splitted_dataset = self.data.train_test_split(test_size = 0.25, seed = 42)

        weights = 1. / np.bincount(splitted_dataset['train']['label'])
        self.class_weight = np.array([weights[l] for l in splitted_dataset['train']['label']])
        self.dataloader = torch.utils.data.DataLoader(splitted_dataset['train'], batch_size = batch_size,
                                                      sampler = torch.utils.data.WeightedRandomSampler(self.class_weight, len(self.class_weight)))
        
        test_weights = 1. / np.bincount(splitted_dataset['test']['label'])
        self.validloader = torch.utils.data.DataLoader(splitted_dataset['test'], batch_size = batch_size,
                                                      sampler = torch.utils.data.WeightedRandomSampler(np.array([test_weights[l] for l in splitted_dataset['test']['label']]),
                                                                                                       len(splitted_dataset['test']['label'])))

    def _clear_cache(self):
         if self.device.type == 'cuda':
            with torch.no_grad(): torch.cuda.empty_cache()
            gc.collect()

    def make_probe(self, prober: torch.nn.Module, enable_grads: bool = False, use_variational: bool = False, layers: list = [1], from_memory = None, save_outputs: bool = False, task_title: str = None) -> np.ndarray:
        """ Main method to do a probing task
            Args:
                prober, callable object: funtion with argumnents __data__ and __labels__ to make probing classification (returns metric value of the task)
                enable_grads, bool: optional flag, whether to propagate grads or not
                                    default = False
                use_variational, bool: optional flag, whether to use variational prober or not
                                      default = False
                layers, list: optional list layers indexes to probe (shoud be 0 < layers < #hiddenLayers)
                              default = [0]
                from_memory, str: optionally load probing data from memory (currently deprecated)
                save_outputs, bool: optional flag, whether to save probing data
                                    default = False
                task_tilte, str: optional way to save probing data, active only if save_outputs = True;
                                default = None
            Returns:
              result, np.ndarray: float array of [#layers, ] output metric results
        """
        assert np.alltrue([l > 0 and l < len(self.fixed_encoder) for l in layers])

        def make_hidden_states(example, model = self.model, device: torch.device = device) -> list:
            """Returns outputs of all model layers 
               Agrs:
                  enable_grads, bool: optional flag, whether to propagate grads or not
                                      default = False
            """
            with torch.no_grad(): 
                output = model(example[0].to(self.device), 
                               attention_mask = example[1].to(self.device),
                               output_hidden_states = True)
            return [hs.cpu().view((len(hs), -1)).numpy() for hs in output.hidden_states]
        
        def _prepare_data(batch):
            """Helper function
            """
            labels = batch['label'].to(self.device)
            inp_values, att_masks =  batch['input_values'][0].to(device), batch['attention_mask'][0].to(self.device)
            #getting ready tp encoder
            with torch.no_grad(): 
                extract_features = self.model.wav2vec2.feature_extractor(inp_values).transpose(1, 2)
                hidden_states, extract_features = self.model.wav2vec2.feature_projection(extract_features)
                att_masks = self.model.wav2vec2._get_feature_vector_attention_mask(extract_features.shape[1], att_masks)
                hidden_states = self.model.wav2vec2._mask_hidden_states(hidden_states, attention_mask = att_masks)

            return hidden_states, att_masks, labels

        if from_memory is not None:
            assert isinstance(from_memory, str)
            raise NotImplementedError("") 
        else:
            print_if_debug("stacking classifiers...", DEBUG)

            loss_fn = Loss(use_variational)

            probing_info = {'loss': [], 'metrics': []}

            inputs, attention_masks, _ = _prepare_data(iter(self.dataloader).next())

            
            model_config = {'in_size': self.model.config.hidden_size * POOLING_TO, 
                            'hidden_size': 100,
                            'out_size': len(set(self.data['label'])),
                            'variational': use_variational}

            for layer in tqdm(layers, total = len(layers)):

                self.model.wav2vec2.encoder.layers = deepcopy(self.fixed_encoder[:layer].to(self.device))     
                for module in self.model.wav2vec2.encoder.layers[:-1]:
                    for param in module.parameters(): param.requires_grad = False

                probing_model = prober(parent_model = self.model.wav2vec2.encoder,
                                       clf = LinearModel(**model_config),
                                       enable_grads = enable_grads
                                       ).to(self.device)
                optim = torch.optim.Adam(probing_model.parameters(), lr = 3. * 1e-3)
                scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, T_max = 10)
                probing_model.eval()
                self.writer.add_graph(probing_model, input_to_model = [inputs, attention_masks])
                
                probing_model.train()
                with self.profiler.profile('train') as prof:
                    for epoch in range(N_EPOCHS):
                        train_loss = []
                        for step, batch in tqdm(enumerate(self.dataloader), total = len(self.dataloader)):
                            inputs, attention_masks, labels = _prepare_data(batch)
                            optim.zero_grad()

                            logits = probing_model(inputs, attention_masks)
                            loss = loss_fn(labels, logits, probing_model.clf)
                            train_loss.append(loss.cpu().item()) 

                            loss.backward()
                            optim.step()
                            prof.step()
                            self._clear_cache()
                        self.writer.add_scalar("training loss of layer {}".format(layer), np.mean(train_loss), epoch * len(self.dataloader))
                        scheduler.step()    
                print_if_debug("validating...", DEBUG)

                if save_outputs:
                    chkpnt = self.checkpointer(probing_model = probing_model.cpu(),
                                               task_title = "" if task_title is None else task_title,
                                               params = model_config, layer_idx = layer, optimizer = optim)
                    print_if_debug("checkpoint {} saved...".format(chkpnt), DEBUG)
                
                self._clear_cache()

                probing_model = probing_model.to(self.device)

                metrics_per_layer, losses_per_layer = [], [] 
                probing_model.eval()
                with self.profiler.profile('validation') as prof:
                    for step, batch in tqdm(enumerate(self.validloader), total = len(self.validloader)):
                        inputs, attention_masks, labels = _prepare_data(batch)
                        with torch.no_grad(): 
                            logits = probing_model(inputs, attention_masks)
                        loss = loss_fn(labels, logits, probing_model.clf)
                        f1 = f1_score(torch.argmax(logits.detach().cpu(), dim = -1).numpy(), labels.detach().cpu().numpy(), average = 'weighted')
                        metrics_per_layer.append(f1)
                        losses_per_layer.append(loss.cpu().item())
                        prof.step()
                        self._clear_cache()
                            
                probing_info['loss'].append(np.mean(losses_per_layer))
                probing_info['metrics'].append(np.mean(metrics_per_layer))

                self.writer.add_scalar("test loss", np.mean(losses_per_layer), layer)
                self.writer.add_scalar("test f1", np.mean(metrics_per_layer), layer)    

               
                probing_model = probing_model.cpu()
                del optim
                del probing_model
                print("\n", self.profiler.monitor_resources(), "\n")

        print_if_debug('running probes...', DEBUG)
        return probing_info

class Probing_pipeline:
    def __init__(self, writer: torch.utils.tensorboard.SummaryWriter,
                 feature: str, model_path: str, data: Dataset = None, lang: str = None, split: str = None) -> None:
        """Hugging Face Dataset wrapper for ASR probing
        Args:
            writer, SummaryWriter: tensorboard writer to debug and visualize all probing process
            feature, str: name of dataset's feature column to make probing onto
            model_path, str: path to model in Hugging Face repo
            custom_feature, bool: optional flag, a possibility to add feature not from original set
            data, Dataset: optional, Hugging Face Dataset class;
                           default = None
            (lang, split), str: optional features to this class to determine necessary dataset options
                                default = None

        """
        log_data.set_verbosity(40 if not DEBUG else 20)
        log_models.set_verbosity(40 if not DEBUG else 20)

        self.writer = writer
        self.lang = lang
        self.split = split
        self.feature = feature
        self.model_path = model_path
        self.dataset = data

    def load_data(self, from_disk: bool, data_path: str = None, own_feature_set: dict = None, only_custom_features: bool = True, **kwargs) -> None:
        """Custom dataloader
        Args:
            from_disk: bool, flag if load data from disk checkpoint or from the Internet
                             default = False
            data_path, str: optional, active only if from_disk = True;
                            default = None      
            own_feature_set, dict (format {feat: int}): optional own mapping to probing labels
                                                    default = None
            only_custom_features, bool:optional flag whether to use only custom features or add "other" ground class,
                                      active only with own_feature_set, default = True
        """
        print_if_debug("loading data...", DEBUG)
        if from_disk:
            assert isinstance(data_path, str) 
            self.dataset =  load_from_disk(data_path, **kwargs)
        elif data_path is not None: 
            assert isinstance(data_path, str) 
            self.dataset = load_dataset(data_path, name = self.lang, split = self.split,
                                        **kwargs)         
        else: assert self.dataset is not None
        self.own_feature_set = own_feature_set; self.only_custom_features = only_custom_features

    def _filter_data(self, own_feature_set: dict, only_custom_features: bool) -> None:
        """
        Args: 
          own_feature_set, dict (format {feat: int}): own mapping to probing labels
                                                    default = None
          only_custom_features, bool: flag whether to use only custom features or add "other" ground class,
                                      active only with own_feature_set, default = True

        """
        self.dataset = self.dataset.filter(lambda example: len(example[self.feature].strip()) > 0)
        if own_feature_set is None: self.f_set = {v: k for k, v in enumerate(list(set(self.dataset[self.feature])))}
        else: 
            assert isinstance(own_feature_set, dict)
            self.f_set = own_feature_set            
            if not only_custom_features:
                self.f_set["other"] = np.max(list(self.f_set.values())) + 1
                def foo(batch):
                    if batch[self.feature] not in self.f_set.keys(): batch[self.feature] = "other"
                    return batch                    
                self.dataset = self.dataset.map(foo)
            else: self.dataset = self.dataset.filter(lambda example: example[self.feature] in self.f_set)


    def get_dataset(self): return self.dataset
    def get_feature_set(self): 
        """All labels """
        return self.f_set

    def preprocess_data(self, preprocessinf_fn: type(lambda x: None), save_path: str = None, drop_columns: list = None, target_processing: type(lambda x: None) = None):
        """
        Args:
            preprocessinf_fn, callable object: prerpocessing dataset function to load all audio, should return the same self.dataset
                                               but with 'speech', 'len_speech', 'sampling_rate' columns
            save_path, str: optional path to save preprocessed data
                            default = None
            drop_columns, list: optional list of string-like columns to drop from the dataset
                                default = None
            target_processing, callable object: prerpocessing dataset function to speech transcript. Shouldn't change the dataset structure
                                                default = None
        """
        print_if_debug("downloading necessary staff...", DEBUG)
        processor =  Wav2Vec2Processor.from_pretrained(self.model_path, cache_dir = CACHE_DIR)
        def encode_labels(example, feature_column: str):
            """Label Encoder
            """
            example["label"] = self.f_set[example[feature_column]]
            return example

        def add_new_features(batch, max_len: int):
            """Preprocessing audio features with padding to maximum lenght"""
            inputs = processor(batch["speech"], sampling_rate = batch["sampling_rate"], return_tensors = "pt", 
                               padding = 'max_length', truncation = 'max_length', max_length = max_len)
            batch['input_values'] = inputs.input_values
            batch['attention_mask'] = inputs.attention_mask
            return batch

        print_if_debug('reading files...', DEBUG)
        if preprocessinf_fn is not None:
            self.dataset = self.dataset.map(preprocessinf_fn, fn_kwargs = {'feature_column': self.feature}, disable_nullable = False)
        print_if_debug('encoding features...', DEBUG)
        self._filter_data(self.own_feature_set, self.only_custom_features)
        self.dataset = self.dataset.map(encode_labels, fn_kwargs = {'feature_column': self.feature})

        print_if_debug('processing features...', DEBUG)
        self.dataset = self.dataset.map(add_new_features, fn_kwargs = {'max_len': np.max(self.dataset['len_speech'])})

        if drop_columns is not None:
            print_if_debug('removing user-picked columns...', DEBUG)
            assert isinstance(drop_columns, list) or isinstance(drop_columns, str)
            if isinstance(drop_columns, str): self.dataset = self.dataset.remove_columns([drop_columns])
            elif isinstance(drop_columns, list): self.dataset = self.dataset.remove_columns(drop_columns)
        self.dataset = self.dataset.remove_columns([self.feature, 'speech', 'len_speech', 'sampling_rate'])

        if target_processing is not None:
            print_if_debug('target processing... (is ON)', DEBUG)
            assert isinstance(target_processing, dict)
            assert ['fn', 'kwargs'] == list(target_processing.keys()) 
            assert isinstance(target_processing['fn'], type(lambda x: None)) and\
                   isinstance(target_processing['kwargs'], dict)

            self.dataset = self.dataset.map(target_processing['fn'], fn_kwargs = target_processing['kwargs'])

        if save_path is not None:
            assert isinstance(save_path, str) 
            print_if_debug('saving files...', DEBUG)
            if self.lang is None: self.lang = "en"
            self.dataset.save_to_disk(save_path + self.feature + "_" + self.lang + "_dataset")
        print('done')
        return self
    
    def run_probing(self, probing_fn, layers: list, enable_grads = False, use_variational: bool = False, init_strategy: str = None, plotting_fn: type(lambda x: None) = None, 
                    save_checkpoints: bool = False, plotting_config: dict = None, **kwargs):
        """Main probing runner
        Args:
           probing_fn, init_strategy -- look at Prober docs
           use_variational, bool: optional flag, whether to use variational prober or not
                                  default = False
           enable_grads, bool: optional flag, whether to propagate grads or not
                               default = False
           plotting_fn: callable, optional way to plot results
                       default = None
           save_checkpoints, bool: an optional flag, whether to save checkpoints
                                   defalult = False
           plotting_config, dict ({"title": str, "metrics": list of used in pro bing fn metrics, "save_path": str}), default = None
        """
        probing_task = Prober(self.model_path, self.writer, data = self.dataset, init_strategy = init_strategy)
        probing_task.get_resources(load_data = False, batch_size = BATCH_SIZE, **kwargs)
       

        probing_results = probing_task.make_probe(probing_fn, use_variational = use_variational, enable_grads = enable_grads, layers = layers, 
                                                  save_outputs = save_checkpoints, task_title = plotting_config['title'])
        
        json.dump({"data": probing_results, "config": plotting_config}, 
                  open(os.path.join(plotting_config['save_path'], plotting_config['title'] + ".json"), 'w' ), cls = NumpyEncoder)
        
        if plotting_fn is not None:
            assert isinstance(plotting_fn,  type(lambda x: None))
            assert isinstance(plotting_config, dict)
            plotting_fn(probing_results, plotting_config)
            plt.show()

        return probing_results

    def cleanup(self):
        """Erasing all cache
        """
        if self.dataset.cleanup_cache_files(): return "succeed"
        else: return "nothing has been deleted"
        
    def disable_cache(self): return set_caching_enabled(False)
    def enable_cache(self): return set_caching_enabled(True)

    def __repr__(self):
        return "Used data: {} \n ".format(self.dataset) +\
                "Used feature {} with set of values = {} \n".format(self.feature, self.f_set) +\
                "Used model: {}".format(self.model_path)

In [5]:
_lang = lambda l: "en" if l is None else l

def make_probing(data, labels):
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.33, shuffle = True, random_state=42)
    clf = SGDClassifier(max_iter = 1000, tol = 1e-2, random_state = 42)
    clf.fit(X_train, y_train)
    # acc = accuracy_score(clf.predict(X_test), y_test)
    f1 = f1_score(clf.predict(X_test), y_test, average = 'weighted')
    del clf
    return f1

##common_voice
def prepare_probing_task(batch, feature_column: str):
    frame_offset, num_frames = 16000, 16000
    sp, sr = load(batch["path"], frame_offset = frame_offset, num_frames = num_frames)
    resampler = transforms.Resample(sr, 16000)
    batch['speech'] = resampler(sp).squeeze().numpy()
    batch["sampling_rate"] = 16000
    batch['len_speech'] = len(batch['speech'])
    return batch


##timit
def prepare_probing_task_timit(batch, feature_column: str):
    frame_offset, num_frames = 16000, 16000
    sp, sr = load(batch["file"], frame_offset = frame_offset, num_frames = num_frames)
    resampler = transforms.Resample(sr, 16000)
    batch['speech'] = resampler(sp).squeeze().numpy()
    batch["sampling_rate"] = 16000
    batch['len_speech'] = len(batch['speech'])
    return batch

metadata = pd.read_csv(TIMIT_METADATA_PATH)
def prepare_probing_task_timit_2(batch, feature_column: str):
    """ Adding new features from  dataset's metadata files
    """
    frame_offset, num_frames = 16000, 16000
    sp, sr = load(batch["file"], frame_offset = frame_offset, num_frames = num_frames)
    resampler = transforms.Resample(sr, 16000)
    batch['speech'] = resampler(sp).squeeze().numpy()
    batch["sampling_rate"] = 16000
    batch['len_speech'] = len(batch['speech'])
    val = metadata[metadata["id"] == batch["speaker_id"]][feature_column].values
    batch[feature_column] = str(val[0]) if len(val) else "other" 
    return batch

def remove_special_characters(batch):
    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower().strip()
    return batch

def plotting_fn(data, config: dict):
    if not DEBUG: clear_output(wait = True)
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_title(config['title'])
    used_metrics = config['metrics']
    if len(used_metrics) == 1: data = np.array(data)[..., np.newaxis]
    assert len(used_metrics) == data.shape[1]
    for ind in range(data.shape[1]): 
        ax.plot(np.arange(0, len(data), 1), data[:, ind], marker = '.', color = 'red', lw = 2, label = used_metrics[ind])
    ax.legend(loc = 'best')
    ax.set_ylabel('metrics'); ax.set_xlabel('#layer')
    ax.grid(True)
    if config['save_path'] is not None:
        assert isinstance(config['save_path'], str)
        pickle.dump(fig, open(config['save_path'] + config['title'] + '.pickle', 'wb'))

In [6]:
class TaskTester:
    def __init__(self, dataset_name: str,
                       features: list,
                       layers: list,
                       dataset_language: list = [None],
                       dataset_split: str = "test",
                       from_disk: bool = False,
                       prefix_data_path: str = None,
                       model_init_strategies: list = [None] + ["full"], 
                       use_variational: bool = False,
                       preprocessing_fn: type(lambda x: None) = prepare_probing_task,
                       probing_fn: torch.nn.Module = ProberModel,
                       enable_grads = False,
                       plotting_fn: type(lambda x: None) = None,
                       save_checkpoints: bool = False, 
                       save_preprocessed_data: str = None,
                       own_feature_set: dict = None,
                       poisoning_ratio: float = 0,
                       poisoning_mapping = None,
                       return_results: bool = False,
                       **kwargs):
        """A wrapper for the whole pipeline
        Agrs:
            dataset_name, str: name of used dataset, if `from_disk`== False, supported only "common_voice" and "timit_asr"
            layers, list: list of layers to probe on (l >= 1 and l <= 24 for l in layers)
            dataset_language, list: a list of supported dataset's languages
                                    deafult (for timit_asr) = [None]
            dataset_split, str: split of the used dataset
                                default = "test"
            from_disk, bool: an optional flag where to take a data for probing (if False, data will be dowlnoaded 
                             from HuggingFace hub)
                             default = False
            prefix_data_path, str: an optinal helpful string to find a dataset on the disk
            model_init_strategies, list: list of probing initializers; supported regims are: 
                                                                                            [None (only pretrained model),
                                                                                             "full" (whole random init.),
                                                                                             "encoder" (only random encoder)]
                                         default = [None, "full"]
            
            use_variational, bool: an optional flag, whether use MDL (True) or on ordinary logistic regression 
                                    default = False
            preprocessing_fn, a callable object: function for extracting audio from dataset
                                                 default = None
            probing_fn, torch.nn.Module class: probing model
                                               defalut = ProberModel
            
            enable_grads, bool: an optional flag, if backprop through any used model or not
                                default = False
            own_feature_set: dict, an optional dict of hand-designed labels for probing,
                                   default = None
            
            return_results: bool: a flag
            save_checkpoints, bool: an optional flag, whether to save checkpoints
                                   defalult = False
            
            other arguments are temporarily deprecated.
                                   
        """
        self.results = []
        layers = list(sorted(layers))
        
        for lang in dataset_language:
            for feature in features:
                for init_strategy in model_init_strategies:
                    title = dataset_name + "_" + _lang(lang) + "_" + feature + "_task_random=" + str(init_strategy) +\
                            "_grads="  +str(enable_grads) + "_variational=" + str(use_variational) + "_poisoned=" + str(poisoning_ratio)
                    writer = SummaryWriter(LOGGING_DIR + title + "/layers={}-{}".format(layers[0], layers[-1]))
                    pipe = Probing_pipeline(writer = writer,
                                            feature = feature, model_path = MODELS_PATH[dataset_name][str(lang)], 
                                            lang = lang, split = dataset_split)
                    pipe.disable_cache()
                    if from_disk: assert isinstance(prefix_data_path, str)
                    pipe.load_data(from_disk = from_disk, data_path = dataset_name if not from_disk else prefix_data_path,
                                   own_feature_set = own_feature_set, only_custom_features = False)
                    print(pipe.preprocess_data(preprocessing_fn, save_path = save_preprocessed_data,
                                               target_processing = None, **kwargs))
                    print("The task title:", title)
                    res = pipe.run_probing(probing_fn, layers = layers, enable_grads = enable_grads, 
                                           use_variational = use_variational, 
                                           plotting_fn = plotting_fn, 
                                           save_checkpoints = save_checkpoints, 
                                           init_strategy = init_strategy, 
                                           plotting_config = {"title": title,
                                                              "custom_features": list(own_feature_set.keys()) if own_feature_set is not None else "None",
                                                              "metrics": ['f1'], 'save_path': os.path.join(GRAPHS_PATH, str(TODAY))},
                                          poisoning_ratio = poisoning_ratio,
                                          poisoning_mapping = poisoning_mapping)
                    pipe.cleanup()
                    if return_results: self.results.append(res)
        if not DEBUG: clear_output(wait = True) 
        !rm /root/.cache/huggingface/* -r
        !rm ./cache/ -r
    def get_testing_results(self):
        """ A format:
        {
        "config": {dict of plotting config including "title"}
        "data": {"loss": [], "metrics": {}}
        }
        """
        return self.results
    def __repr__(self): return "ez4ence"

In [7]:
layers = np.arange(1, 24, 1)
# TaskTester(dataset_name = "timit_asr",
#            features = ['sex', 'age_bin'],
#            layers = layers,
#            preprocessing_fn = prepare_probing_task_timit_2,
#            use_variational = False,
#            enable_grads = False,
#            probing_fn = ProberModel,
#            save_checkpoints = False,
#            poisoning_ratio = 0,
#            drop_columns = ['word_detail', 'phonetic_detail'])

In [ ]:
TaskTester(dataset_name = "timit_asr",
           features = ['sex', 'age_bin'],
           layers = layers,
           preprocessing_fn = prepare_probing_task_timit_2,
           use_variational = True,
           enable_grads = False,
           probing_fn = ProberModel,
           save_checkpoints = False,
           poisoning_ratio = 0,
           drop_columns = ['word_detail', 'phonetic_detail'])

In [ ]:
layers = [1, 2, 3, 5, 7, 11, 17]
TaskTester(dataset_name = "timit_asr",
           features = ['sex'],
           layers = layers,
           preprocessing_fn = prepare_probing_task_timit_2,
           use_variational = True,
           enable_grads = False,
           probing_fn = ProberModel,
           save_checkpoints = False,
           model_init_strategies = [None],
           poisoning_ratio = .67,
           drop_columns = ['word_detail', 'phonetic_detail'])

TaskTester(dataset_name = "timit_asr",
           features = ['sex'],
           layers = layers,
           preprocessing_fn = prepare_probing_task_timit_2,
           use_variational = True,
           enable_grads = False,
           probing_fn = ProberModel,
           save_checkpoints = False,
           model_init_strategies = [None],
           poisoning_ratio = .0,
           drop_columns = ['word_detail', 'phonetic_detail'])

TaskTester(dataset_name = "timit_asr",
           features = ['pos_tag'],
           layers = layers,
           prefix_data_path='./pos_tag_set/',
           preprocessing_fn = None,
           use_variational = True,
           enable_grads = False,
           save_checkpoints = False,
           model_init_strategies = [None],
           poisoning_ratio = .67,
           probing_fn = ProberModel,
           from_disk=True)

In [ ]:
# TaskTester(dataset_name = "timit_asr",
#            features = ['sex', 'age_bin'],
#            layers = [4] + [12] + [18],
#            preprocessing_fn = prepare_probing_task_timit_2,
#            use_variational = False,
#            enable_grads = True,
#            probing_fn = ProberModel,
#            save_checkpoints = True,
#            drop_columns = ['word_detail', 'phonetic_detail'])
# TaskTester(dataset_name = "timit_asr",
#            features = ['sex', 'age_bin'],
#            layers = [4] + [12] + [18],
#            preprocessing_fn = prepare_probing_task_timit_2,
#            use_variational = False,
#            enable_grads = False,
#            probing_fn = ProberModel,
#            save_checkpoints = True,
#            drop_columns = ['word_detail', 'phonetic_detail'])

# TaskTester(dataset_name = "timit_asr",
#            features = ['sex', 'age_bin'],
#            layers = [4] + [12] + [18],
#            preprocessing_fn = prepare_probing_task_timit_2,
#            use_variational = True,
#            enable_grads = True,
#            save_checkpoints = True,
#            probing_fn = ProberModel,
#            drop_columns = ['word_detail', 'phonetic_detail'])
# TaskTester(dataset_name = "timit_asr",
#            features = ['sex', 'age_bin'],
#            layers = [4] + [12] + [18],
#            preprocessing_fn = prepare_probing_task_timit_2,
#            use_variational = True,
#            enable_grads = False,
#            save_checkpoints = True,
#            probing_fn = ProberModel,
#            drop_columns = ['word_detail', 'phonetic_detail'])

# По информации о фонетических|лингвистических признаках


In [ ]:
modes = ['word_detail', 'phonetic_detail']
class LinguisticDataset:
    def __init__(self, mode: str, save_path: str = None):
        """Linguistic dataset wrapper for TIMIT_ASR dataset
        Args:
             mode, str: 'word_detail' or 'phonetic_detail'
             save_path, str: optional way to save probing data, active only if save_outputs = True;
                            default = None
        """
        assert mode in ['word_detail', 'phonetic_detail']
        self.mode = mode
        self.save_path = save_path
        self.D = load_dataset('timit_asr', split = 'test')
        self.data = []
    def format_dataset(self):
        """Making JSON for TIMIT_ASR dataset
        """
        def process_fn(batch):
            data = {'long': []}

            for i in range(len(batch[self.mode]['start'])): data['long'].append((batch[self.mode]['start'][i], 
                                                                                  batch[self.mode]['stop'][i]))
            data['path'] = batch['file']
            data['utterance'] = batch[self.mode]['utterance']
            return data
        print_if_debug('making json...',DEBUG)
        for batch in self.D: self.data.append(process_fn(batch)) 
        if self.save_path is not None: assert isinstance(self.save_path, str)
        else: self.save_path = '.'   
        with open(self.save_path + 'timit_'+ self.mode +'.json', 'w') as fout: json.dump(self.data, fout)

    def __call__(self, additional_preprocessing: type(lambda x: None) = None, debug: bool = False, take_n: int = None) -> Dataset:
        """
        Args:
            additional_preprocessing, callable object: function to make new labels from phonemes, eg. POS-tags
                                                      default  = None, 
            debug, bool: flag, default = False
            take_n, int: how many samples from original set to take
                        default   = 10000 (None)
        Returns: new Hugging Face dataset
        """
        self.format_dataset()
        dataset = Dataset.from_pandas(pd.read_json(self.save_path + 'timit_'+ self.mode +'.json'))
        def mapping_fn(batch):
            """Phonemes mapping
            """
            speech_array, sampling_rate = load(batch["path"])
            resampler = transforms.Resample(sampling_rate, 16000)
            batch["speech"] = speech_array.squeeze().numpy()
            batch['atoms'] = []
            for l in batch['long']: 
                batch['atoms'].append(batch['speech'][l[0]:l[1]])
            return batch

        dataset = dataset.map(mapping_fn)
        new_data = []
        for example in dataset:
            for ind in range(len(example['atoms'])):
              if  len(example['atoms'][ind]) > 0 and \
                  len(example['atoms'][ind]) <= MAX_LEN if self.mode == 'word_detail' else int(0.01 * MAX_LEN): 

                  new_data.append([example['atoms'][ind], 16000,
                                   len(example['atoms'][ind]),
                                   example['utterance'][ind]])

              else: print_if_debug(example['utterance'][ind] + " " + "{}".format(len(example['atoms'][ind])), debug)
        new_dataset = pd.DataFrame(new_data, columns = ['speech', 'sampling_rate', 'len_speech', 'utterance'])

        if take_n is not None: 
            assert isinstance(take_n, int)
            print_if_debug("taking a slice of {} elements".format(take_n), debug)
            new_dataset = new_dataset.sample(n = take_n, random_state = 42)

        new_dataset =  Dataset.from_pandas(new_dataset)
        if additional_preprocessing is not None:
            assert isinstance(additional_preprocessing, type(lambda x: None))
            print_if_debug("adding new features...", debug)
            new_dataset = new_dataset.map(additional_preprocessing, batched = False)
        return new_dataset

In [ ]:
def pos_tagging_fn(batch):
    batch['pos_tag'] = nltk.pos_tag([batch['utterance']])[0][1]
    return batch

def random_labeling_fn(batch, label: str = 'pos_tag', n_classes: int = 10):
    batch[label + "_random"] = str(np.random.randint(0, n_classes))
    return batch

def label_batch(batch):
    batch = pos_tagging_fn(batch)
    batch = random_labeling_fn(batch)
    return batch


In [ ]:
# lingusitic_dataset = DatasetDict()
# for mode in modes:
#     lingusitic_dataset[mode] = LinguisticDataset(mode, 
#                                        save_path = ".")(
#                                            label_batch if mode == 'word_detail' else None,
#                                            debug = DEBUG,
#                                            take_n = None if mode == 'word_detail' else 15000 )

# lingusitic_dataset['word_detail'].save_to_disk("./pos_tag_set/")
# lingusitic_dataset['phonetic_detail'].save_to_disk("./phonetic_set/")
# del lingusitic_dataset


## POS tag 

In [ ]:
# TaskTester(dataset_name = "timit_asr",
#            features = ['pos_tag'],
#            layers = [4] + [12] + [18],
#            prefix_data_path='./pos_tag_set/',
#            preprocessing_fn = None,
#            use_variational = False,
#            enable_grads = True,
#            save_checkpoints = True,
#            probing_fn = ProberModel,
#            from_disk=True)

# TaskTester(dataset_name = "timit_asr",
#            features = ['pos_tag'],
#            layers = [4] + [12] + [18],
#            prefix_data_path='./pos_tag_set/',
#            preprocessing_fn = None,
#            use_variational = True,
#            enable_grads = True,
#            save_checkpoints = True,
#            probing_fn = ProberModel,
#            from_disk=True)

# TaskTester(dataset_name = "timit_asr",
#            features = ['pos_tag'],
#            layers = [4] + [12] + [18],
#            prefix_data_path='./pos_tag_set/',
#            preprocessing_fn = None,
#            use_variational = False,
#            enable_grads = False,
#            save_checkpoints = True,
#            probing_fn = ProberModel,
#            from_disk=True)

# TaskTester(dataset_name = "timit_asr",
#            features = ['pos_tag'],
#            layers = [4] + [12] + [18],
#            prefix_data_path='./pos_tag_set/',
#            preprocessing_fn = None,
#            use_variational = True,
#            enable_grads = False,
#            save_checkpoints = True,
#            probing_fn = ProberModel,
#            from_disk=True)

## Простые фонемы

In [8]:
stops =  ["b", "d", "g", "p", "t", "k", "dx", "q"]  
closed_stops = ["bcl", "dcl", "gcl", "pcl", "tck", "kcl"]   
####################################################################################
vowels = ["iy", "ih", "eh", "ey", "ae", "aa", "aw", "ay", "ah", "ao", "oy", "ow",
          "uh", "uw", "ux", "er", "ax", "ix", "axr", "ax-h"]
####################################################################################
others = ["pau", "h#"]

####################################################################################
####################################################################################
f_set = lambda col: {v: k for k, v in enumerate(list(set(col)))}

In [9]:
# TaskTester(dataset_name = "timit_asr",
#            features = ['utterance'],
#            layers = [4] + [12] + [18],
#            prefix_data_path='./phonetic_set/',
#            preprocessing_fn = None,
#            use_variational = False,
#            enable_grads = True,
#            save_checkpoints = True,
#            probing_fn = ProberModel,
#            own_feature_set = f_set(vowels), #f_set(vowels) for only vowels classification 
#            from_disk=True)

# TaskTester(dataset_name = "timit_asr",
#            features = ['utterance'],
#            layers = [4] + [12] + [18],
#            prefix_data_path='./phonetic_set/',
#            preprocessing_fn = None,
#            use_variational = True,
#            enable_grads = True,
#            save_checkpoints = True,
#            probing_fn = ProberModel,
#            own_feature_set = f_set(vowels), 
#            from_disk=True)

# TaskTester(dataset_name = "timit_asr",
#            features = ['utterance'],
#            layers = [4] + [12] + [18],
#            prefix_data_path='./phonetic_set/',
#            preprocessing_fn = None,
#            use_variational = False,
#            enable_grads = False,
#            save_checkpoints = True,
#            probing_fn = ProberModel,
#            own_feature_set = f_set(vowels), #f_set(vowels) for only vowels classification 
#            from_disk=True)

# TaskTester(dataset_name = "timit_asr",
#            features = ['utterance'],
#            layers = [4] + [12] + [18],
#            prefix_data_path='./phonetic_set/',
#            preprocessing_fn = None,
#            use_variational = True,
#            enable_grads = False,
#            save_checkpoints = True,
#            probing_fn = ProberModel,
#            own_feature_set = f_set(vowels), 
#            from_disk=True)

In [ ]:
# TaskTester(dataset_name = "timit_asr",
#            features = ['utterance'],
#            layers = np.arange(1, 24, 1),
#            prefix_data_path='./phonetic_set/',
#            preprocessing_fn = None,
#            use_variational = True,
#            enable_grads = False,
#            save_checkpoints = False,
#            poisoning_ratio = 0.0,
#            probing_fn = ProberModel,
#            own_feature_set = f_set(vowels), 
#            from_disk=True)


TaskTester(dataset_name = "timit_asr",
           features = ['utterance'],
           layers = np.arange(1, 24, 1),
           prefix_data_path='./phonetic_set/',
           preprocessing_fn = None,
           use_variational = False,
           enable_grads = False,
           save_checkpoints = False,
           poisoning_ratio = 0.0,
           probing_fn = ProberModel,
           own_feature_set = f_set(vowels), 
           from_disk=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/15000 [00:00<?, ?ex/s]

  0%|          | 0/15000 [00:00<?, ?ex/s]

  0%|          | 0/15000 [00:00<?, ?ex/s]

done
Used data: Dataset({
    features: ['__index_level_0__', 'label', 'input_values', 'attention_mask'],
    num_rows: 15000
}) 
 Used feature utterance with set of values = {'ey': 0, 'uw': 1, 'iy': 2, 'aw': 3, 'ax': 4, 'ae': 5, 'aa': 6, 'ux': 7, 'oy': 8, 'ay': 9, 'eh': 10, 'ow': 11, 'ix': 12, 'ah': 13, 'uh': 14, 'ao': 15, 'axr': 16, 'er': 17, 'ih': 18, 'ax-h': 19, 'other': 20} 
Used model: elgeish/wav2vec2-large-lv60-timit-asr
The task title: timit_asr_en_utterance_task_random=None_grads=False_variational=False_poisoned=0.0


...

  0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/225 [00:00<?, ?it/s]

  0%|          | 0/225 [00:00<?, ?it/s]

In [ ]:
#plotting cell!
def get_data(fl):
    with open(fl) as json_file:
        data = json.load(json_file)
    return data

dataset = 'timit_asr'
feature = 'sex'
lang = 'en'

rnd = get_data(GRAPHS_PATH + '2022-01-28/' + f"{dataset}_{lang}_{feature}_task_random=full_grads=False_variational={str(False)}_poisoned=0.json")
d = get_data(GRAPHS_PATH + '2022-01-28/' + f"{dataset}_{lang}_{feature}_task_random={str(None)}_grads=False_variational={str(False)}_poisoned=0.json")
var_rnd = get_data(GRAPHS_PATH + '/2022-01-28/' + f"{dataset}_{lang}_{feature}_task_random=full_grads=False_variational={str(True)}_poisoned=0.json")
var_d = get_data(GRAPHS_PATH  + '2022-01-28/' + f"{dataset}_{lang}_{feature}_task_random={str(None)}_grads=False_variational={str(True)}_poisoned=0.json")

# x =  [1, 2, 3] + [8, 11, 13] + [21, 22, 23] 
#on linguistic features:
#x = [1, 2, 3] + [8, 11, 13] + [21, 22]

def plot(x, rnd, d, var_rnd, var_d):
    print(x)
    print("var. ord. / data. ord.", np.array(var_d['data']['loss']) / np.array(d['data']['loss']))
    print("data. ord. / data. rand.", np.array(var_rnd['data']['loss']) / np.array(rnd['data']['loss']))

    print("data. rand. / data. ord.", np.array(rnd['data']['loss']) / np.array(d['data']['loss']))
    print("var. rnd. / var. ord.", np.array(var_rnd['data']['loss']) / np.array(var_d['data']['loss']))

    fig, axs = plt.subplots(2, 2, sharex=True)
    plt.rcParams.update({'font.size': 14})
    plt.rcParams["figure.figsize"] = (16, 12)

    axs[0, 0].bar(x, var_d['data']['loss'], color = 'r', label = 'variational')
    axs[0, 0].bar(x, d['data']['loss'], color = 'k', label = 'data')
    axs[0, 0].set_title("Codelength on pretrained network")
    axs[0, 0].set_xticks(x)
    axs[0, 0].legend()
    axs[0, 0].grid(True)

    axs[1, 0].bar(x, var_rnd['data']['loss'], color = 'r', label = 'variational')
    axs[1, 0].bar(x, var_d['data']['loss'], color = 'y', label = 'ord. variational')
    axs[1, 0].bar(x, rnd['data']['loss'], color = 'k', label = 'data')
    axs[1, 0].set_title("Codelength on rand. init. network")
    axs[1, 0].legend()
    axs[1, 0].grid(True)

    axs[0, 1].plot(x, d['data']['metrics'],  lw = 2, label = 'pretrained')
    axs[0, 1].plot(x, rnd['data']['metrics'],  lw = 2, label = 'rand. init.')
    axs[0, 1].legend()
    axs[0, 1].grid(True)
    axs[0, 1].set_title("Metrics [F1]")

    axs[1, 1].plot(x, np.abs(np.array(d['data']['metrics']) - np.array(rnd['data']['metrics'])), c = 'k', lw = 2, label = 'data')
    axs[1, 1].plot(x, np.abs(np.array(var_d['data']['metrics']) - np.array(var_rnd['data']['metrics'])), c = 'r', lw = 2, label = 'variational')
    axs[1, 1].set_title("Selectivity " + r"$\sigma(m, r, \hat{y}, y) \triangleq |r(m, \hat{y}, y) - r(m_{rand}, \hat{y}, y)|$")
    axs[1, 1].legend()
    axs[1, 1].grid(True)
    fig.tight_layout()

    plt.savefig(f'{feature}.png', bbox_inches='tight')

plot(np.arange(1, 24, 1), rnd, d, var_rnd, var_d)

In [ ]:
dataset = 'timit_asr'
feature = 'pos_tag'
lang = 'en'

rnd = get_data(GRAPHS_PATH + '2022-01-28/' + f"{dataset}_{lang}_{feature}_task_random=full_grads=False_variational={str(False)}_poisoned=0.json")
d = get_data(GRAPHS_PATH + '2022-01-28/' + f"{dataset}_{lang}_{feature}_task_random={str(None)}_grads=False_variational={str(False)}_poisoned=0.json")
var_rnd = get_data(GRAPHS_PATH + '/2022-01-28/' + f"{dataset}_{lang}_{feature}_task_random=full_grads=False_variational={str(True)}_poisoned=0.json")
var_d = get_data(GRAPHS_PATH  + '2022-01-28/' + f"{dataset}_{lang}_{feature}_task_random={str(None)}_grads=False_variational={str(True)}_poisoned=0.json")
plot(np.arange(1, 24, 1), rnd, d, var_rnd, var_d)

In [ ]:
dataset = 'timit_asr'
feature = 'age_bin'
lang = 'en'

rnd = get_data(GRAPHS_PATH + '2022-01-28/' + f"{dataset}_{lang}_{feature}_task_random=full_grads=False_variational={str(False)}_poisoned=0.json")
d = get_data(GRAPHS_PATH + '2022-01-28/' + f"{dataset}_{lang}_{feature}_task_random={str(None)}_grads=False_variational={str(False)}_poisoned=0.json")
var_rnd = get_data(GRAPHS_PATH + '/2022-01-28/' + f"{dataset}_{lang}_{feature}_task_random=full_grads=False_variational={str(True)}_poisoned=0.json")
var_d = get_data(GRAPHS_PATH  + '2022-01-28/' + f"{dataset}_{lang}_{feature}_task_random={str(None)}_grads=False_variational={str(True)}_poisoned=0.json")
plot(np.arange(1, 24, 1), rnd, d, var_rnd, var_d)

In [ ]:
dataset = 'timit_asr'
feature = 'utterance'
lang = 'en'

rnd = get_data(GRAPHS_PATH + '2022-01-28/' + f"{dataset}_{lang}_{feature}_task_random=full_grads=False_variational={str(False)}_poisoned=0.json")
d = get_data(GRAPHS_PATH + '2022-01-28/' + f"{dataset}_{lang}_{feature}_task_random={str(None)}_grads=False_variational={str(False)}_poisoned=0.json")
var_rnd = get_data(GRAPHS_PATH + '/2022-01-28/' + f"{dataset}_{lang}_{feature}_task_random=full_grads=False_variational={str(True)}_poisoned=0.json")
var_d = get_data(GRAPHS_PATH  + '2022-01-28/' + f"{dataset}_{lang}_{feature}_task_random={str(None)}_grads=False_variational={str(True)}_poisoned=0.json")
plot(np.arange(1, 24, 1), rnd, d, var_rnd, var_d)

## Inference class (TBD)

In [ ]:
class InferenceR:
    def __init__(self, probing_model_checkpoint, model_path: str,
                       on_logits: bool = False,
                       on_weights: bool = False,
                       on_attentions: bool = True):
        """An inference class
        """
        self.checkpoint = torch.load(probing_model_checkpoint)
        self.model = Wav2Vec2ForCTC.from_pretrained(model_path, cache_dir = CACHE_DIR)
        self.probe = Wav2Vec2ForCTC.from_pretrained(model_path, cache_dir = CACHE_DIR)

        #flags
        self.on_l = on_logits
        self.on_w = on_weights
        self.on_att = on_attentions

    def _reconstruct_model(self, prober):
        layer_idx = self.checkpoint['layer_index']
        self.probe.wav2vec2.encoder.layers = self.probe.wav2vec2.encoder.layers[:layer_idx]
        self.probing_model = prober(parent_model = self.probe.wav2vec2.encoder,
                                    clf = LinearModel(**self.checkpoint['model_params']),
                                    enable_grads = False)
        self.probing_model.load_state_dict(self.checkpoint['model_state_dict'])
        self.probe.wav2vec2.encoder.layers = self.model.wav2vec2.encoder.layers[layer_idx:]
        self.clf = self.probing_model.clf
    def _compute_attention_flow(self, dataloader: torch.utils.data.DataLoader):
        pass
    def _compute_shap_cam_on_logits(self, on_target: bool = True, on_probing: bool = False): 
        pass
    def visualize_attention_flow(self, data):
        pass
    def visualize_shap_cam(self, data, tgt: str):
        pass